In [ ]:
!pip install qiskit

from IPython.display import clear_output
clear_output(wait=False)

In [ ]:
from qiskit import *
import random
import numpy as np
from scipy.linalg import expm
from qiskit.visualization import plot_histogram
from qiskit.visualization import plot_state_qsphere
from qiskit.visualization import plot_state_paulivec
from datetime import datetime

In [ ]:
start_time = datetime.now()

H = [[0, 1],                                  #Operator whose eigenvalues and eigenvectors are to be found 
    [1,0]] 
H = np.matrix(H)  
print("Hamiltonian matrix: \n", H)
print("--------------------------------------------------------")

t = 1                                    
U = expm(-1j * H * t)

param_theta = random.random() * np.pi        #Angle for Unitary gate A(Capital theta) Capital_theta=(param_theta, param_phi)
param_phi = random.random() * 2 * np.pi

alpha = 0.5                                  #Parameters for weight w
beta = 0.6

Ms = 0 
Mh = 200

sim = Aer.get_backend('qasm_simulator')
svsim = Aer.get_backend('statevector_simulator')

while Ms < Mh : 
  #Unitary gate A(capital_theta) acting on 2nd qubit, phi 
  qr = QuantumRegister(1)
  cr = ClassicalRegister(1)
  eig_vector = QuantumCircuit(qr,cr)             #Circuit for finding eigvector 
  eig_vector.u3(param_theta, param_phi, 0, qr[0])
  eig_vector.squ(U,qr[0])
  eig_vector.u3(-param_theta,0, -param_phi,qr[0])


  eig_vector.measure(0,0)
  res_vec = execute(eig_vector, backend = sim, shots =1).result()
  res_vec_dict = res_vec.get_counts()
  #print(res_vec_dict)
  res_vec_collapsed = int(list(res_vec_dict.keys())[list(res_vec_dict.values()).index(1)])
  #print(res_vec_collapsed)

  if res_vec_collapsed == 0 : 
    #print("success")
    Ms = Ms + 1 

  elif res_vec_collapsed == 1 : 
    #print("fail")
    w = alpha * (Ms + 1)**(-beta)
    r1 = random.choice((-1, 1)) * random.random() * np.pi
    r2 = random.choice((-1, 1)) * random.random() * 2* np.pi 
    Ms = 0 
    param_theta = (param_theta + w * r1) % (np.pi)
    param_phi = (param_phi + w * r2) % (2*np.pi)

qr = QuantumRegister(1)
final_eig_vector = QuantumCircuit(qr)
#final_eig_vector.h(qr[0])
final_eig_vector.u3(param_theta, param_phi, 0, qr[0])
final_eig_vector.squ(U,qr[0])
final_eig_vector.u3(-param_theta,0, -param_phi,qr[0])

print("Final Circuit for finding Eigvector : A(capital_theta)| U | A_dagger(capital_theta) ==> |0> \n")
print(final_eig_vector)
print("-----------------------------------------------------------")



result_fiducial = execute(final_eig_vector, backend = svsim).result()
counts_fiducial = result_fiducial.get_counts()
#print(counts_fiducial)
res_sv = result_fiducial.get_statevector(final_eig_vector)
print("\nFiducial State: Simulation ",res_sv)


estimate = QuantumCircuit(1)
estimate.u3(param_theta, param_phi, 0, 0)
estimated_state = execute(estimate, backend = svsim).result().get_statevector(estimate)
estimated_state = np.matrix(estimated_state)


print("Estimated State:",estimated_state)


"**********************Input the theoretically calculated eigenstate************************"
#eigstate = [-1/np.sqrt(2),1/np.sqrt(2)]
#eigstate = np.matrix(eigstate)
"*******************************************************************************************"

#fidelity = (np.abs(eigstate.conjugate() * estimated_state.transpose()))**2
#print("Fidelity :" ,fidelity[0,0])
#print("Error:", 1 - fidelity[0,0])
#print("1/200 =", 1/200)

 
#print("Eigenstate:", eigstate)

end_time = datetime.now()
runtime = str(end_time - start_time)
print(runtime)


Hamiltonian matrix: 
 [[0 1]
 [1 0]]
--------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).


Final Circuit for finding Eigvector : A(capital_theta)| U | A_dagger(capital_theta) ==> |0> 

         ┌─────────────────────┐┌─────┐┌───────────────────────┐
q229202: ┤ U3(1.5215,6.2126,0) ├┤ Squ ├┤ U3(-1.5215,0,-6.2126) ├
         └─────────────────────┘└─────┘└───────────────────────┘
-----------------------------------------------------------

Fiducial State: Simulation  Statevector([0.54030231-0.83835309j, 0.05936129-0.04139748j],
            dims=(2,))
Estimated State: [[0.72433398+0.j         0.68773158-0.04863697j]]
The two eigstates are:
[[ 0.72970854+0.j        -0.68079173-0.0636244j]]
[[ 0.67203157+0.j         -0.74052145+0.00124639j]]
0:00:40.651048


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:75: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be

In [ ]:
# This part is just to verify the working of exponential hamiltonian gate. 


qr_test = QuantumRegister(1)
test = QuantumCircuit(qr_test)
test.squ(U,qr_test[0])
res_test = execute(test, backend = svsim).result().get_statevector(test)
print("Vector after exponential hamiltonian applied : Simulation : \n",res_test)

print("---------------------------------------------------------")

#Verification of exponential hamiltonian for H = X
circ = QuantumCircuit(1)
circ.rz(2,0)
circ_sv = execute(circ, backend = svsim).result().get_statevector(circ)
print("Vector after exponential hamiltonian is applied : Theoretical\n",circ_sv)

if res_test.all() == circ_sv.all() :
  print("\n\nVerified")

Vector after exponential hamiltonian applied : Simulation : 
 Statevector([ 5.40302306e-01+0.j        , -3.22162947e-16-0.84147098j],
            dims=(2,))
---------------------------------------------------------
Vector after exponential hamiltonian is applied : Theoretical
 Statevector([0.54030231-0.84147098j, 0.        +0.j        ],
            dims=(2,))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: The return type of saved statevectors has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Statevector` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
